# Web Mining and Applied NLP (44-620)

## Requests, JSON, and NLP

### Student Name: Courtney Pigford S560274

https://github.com/cboss320/Module_4_Assignment

Perform the tasks described in the Markdown cells below.  When you have completed the assignment make sure your code cells have all been run (and have output beneath them) and ensure you have committed and pushed ALL of your changes to your assignment repository.

Make sure you have [installed spaCy and its pipeline](https://spacy.io/usage#quickstart) and [spaCyTextBlob](https://spacy.io/universe/project/spacy-textblob)

Every question that requires you to write code will have a code cell underneath it; you may either write your entire solution in that cell or write it in a python file (`.py`), then import and run the appropriate code to answer the question.

This assignment requires that you write additional files (either JSON or pickle files); make sure to submit those files in your repository as well.

1. The following code accesses the [lyrics.ovh](https://lyricsovh.docs.apiary.io/#reference/0/lyrics-of-a-song/search) public api, searches for the lyrics of a song, and stores it in a dictionary object.  Write the resulting json to a file (either a JSON file or a pickle file; you choose). You will read in the contents of this file for future questions so we do not need to frequently access the API.

In [36]:
import json
import pickle 

import requests 
import spacy 
from spacytextblob.spacytextblob import SpacyTextBlob

result = json.loads(requests.get('https://api.lyrics.ovh/v1/They Might Be Giants/Birdhouse in your soul').text)

In [62]:
import json
result = {"artist": "they might be giants", "title": "birdhouse in your soul", "lyrics": "lyrics of the song"} 
with open ('lyrics.json', 'w') as file:
    json.dump(result, file)

2. Read in the contents of your file.  Print the lyrics of the song (not the entire dictionary!) and use spaCyTextBlob to perform sentiment analysis on the lyrics.  Print the polarity score of the sentiment analysis.  Given that the range of the polarity score is `[-1.0,1.0]` which corresponds to how positive or negative the text in question is, do you think the lyrics have a more positive or negative connotaion?  Answer this question in a comment in your code cell.

In [38]:
import json
import spacy 
from spacytextblob.spacytextblob import SpacyTextBlob

with open('lyrics.json', 'r') as file: 
    data = json.load(file)
lyrics = data["lyrics"]

print("Lyrics:")
print(lyrics)

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("spacytextblob")
doc = nlp(lyrics)
polarity_score = doc._.polarity
print("Polarity Score:", polarity_score)


Lyrics:
lyrics of the song
Polarity Score: 0.0


3. Write a function that takes an artist, song, and filename, accesses the lyrics.ovh api to get the song lyrics, and writes the results to the specified filename.  Test this function by getting the lyrics to any four songs of your choice and storing them in different files.

In [39]:
import lyricsgenius as lg
genuis = lg.Genius(api_key, skip_non_songs=True, excluded_terms=["(Remix)", "(Live)"], remove_section_headers=True)
artist = genuis.search_artist("Birdhouse in your soul",max_songs=4)

Searching for songs by Birdhouse in your soul...

No results found for 'Birdhouse in your soul'.


In [54]:
import lyricsgenius as lg 
import json

def get_genius_access_token():
    return os.getenv("genius_access_token")

def get_song_lyrics(artist, song, filename):
    access_token = get_genius_access_token() 
    genuis = lg.Genius(api_key, skip_non_songs=True, excluded_terms=["(Remix)", "(Live)"], remove_section_headers=True)
    try:
        song=genius.search_song(song, artist)
        if song: 
            lyrics = song.lyrics
            with open(filename, 'w')as file: 
                json.dump(lyrics, file) 
            print(f"{song.artist}_lyrics_of_{song.title}_song saved as {filename}")
        else:
            print(f"failed to get lyrics")
    except Exception as e:
        print(f"no lyrics for {artist}:{str(e)}")

get_song_lyrics("Ashanti", "Happy", "happy_lyrics.json")
get_song_lyrics("Ashanti", "Foolish", "foolish_lyrics.json")
get_song_lyrics("Ashanti", "Baby", "baby_lyrics.json")
get_song_lyrics("Ashanti", "Rain On Me", "rain_on_me_lyrics.json")

Searching for "Happy" by Ashanti...
Done.
Ashanti_lyrics_of_Happy_song saved as happy_lyrics.json
Searching for "Foolish" by Ashanti...
Done.
Ashanti_lyrics_of_Foolish_song saved as foolish_lyrics.json
Searching for "Baby" by Ashanti...
Done.
Ashanti_lyrics_of_Baby_song saved as baby_lyrics.json
Searching for "Rain On Me" by Ashanti...
Done.
Ashanti_lyrics_of_Rain on Me_song saved as rain_on_me_lyrics.json


4. Write a function that takes the name of a file that contains song lyrics, loads the file, performs sentiment analysis, and returns the polarity score.  Use this function to print the polarity scores (with the name of the song) of the three files you created in question 3.  Does the reported polarity match your understanding of the song's lyrics? Why or why not do you think that might be?  Answer the questions in either a comment in the code cell or a markdown cell under the code cell.

In [61]:
import json
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

def analyze_lyrics(filename):
    nlp = spacy.load('en_core_web_sm')
    nlp.add_pipe('spacytextblob')
    with open(filename, 'r') as file:
        lyrics = json.load(file)
    doc = nlp(lyrics)
    polarity = doc._.polarity
    
    return polarity

files = ["happy_lyrics.json", "foolish_lyrics.json", "baby_lyrics.json", "rain_on_me_lyrics.json"]
for file in files: 
    polarity = analyze_lyrics(file)
    print(f"Polarity of {file}: {polarity}")

Polarity of happy_lyrics.json: 0.37004517221908534
Polarity of foolish_lyrics.json: -0.21999999999999997
Polarity of baby_lyrics.json: 0.18857709750566892
Polarity of rain_on_me_lyrics.json: 0.01991074768852546


The polarity for Happy, Baby and Rain on Me all are more positive songs so being in the postive makes sense. Foolish, I can see how it has a negavtive polarity since it does talk about the faults of one person. 